In [1]:
import data_utils
import pred_utils
import models

import numpy as np
import os
import json

SEED: 13111996
python 3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
keras version 2.4.0
tensorflow version 2.3.0


In [3]:
PATH = 'storage'
ARTIST = "zun"
REP = 1

NOTES_WINDOW = 8
CHORDS_WINDOW = 3
PITCHES_PER_CHORD = 4
MODE = "embeddings"

MIN_FREQ = 5 * REP
UNKNOWN = "UNK"

VALIDATION_SIZE = 0.1

data = data_utils.get_parsed_data(PATH)
chords_mapping = data_utils.get_chords_mapping(PATH)

NONECLASS = False
indices_cache = {}

In [4]:
if MODE == 'embeddings' or MODE == "custom":
    EMBEDDINGS = data_utils.load_embeddings(PATH, embeddings_dir="embeddings")
    NOTES_EMB_SIZE = EMBEDDINGS["NOTES"].vector_size
    CHORDS_EMB_SIZE = EMBEDDINGS["CHORDS"].vector_size
else:
    EMBEDDINGS = None

> Load notes embeddings: embeddings_NOTES_size75_window8_CBOW_H-SFMAX.bin
> Load chords embeddings: embeddings_CHORDS-SHORT_size100_window2_CBOW_NEG-S.bin


In [5]:
TEST_SELECT = data_utils.select_test(data, PATH)
TRAIN_SELECT = data_utils.select_train(data, artist=ARTIST, test_select=TEST_SELECT, chorus_verse_only=not isinstance(ARTIST, list))

TRAIN_SONGS, notes_translated, chords_translated = data_utils.select_songs(data, TRAIN_SELECT, REPEAT=1)
TEST_SONGS, notes_translated_test, chords_translated_test = data_utils.select_songs(data, TEST_SELECT, REPEAT=1)
print(len(TRAIN_SONGS), len(TEST_SONGS))

X_train_n_raw, X_train_ch_raw, y_train_raw, is_start_train, row_to_song_idx_train = data_utils.prepare_raw_X_y(
    notes_translated, 
    chords_translated,
    notes_window=NOTES_WINDOW,
    chords_window=CHORDS_WINDOW,
    use_next_chord=True,
    rep=REP
)

y_train_raw_pitches, y_train_raw_durations = data_utils._split_notes(y_train_raw)
y_train_raw_pitches, y_train_raw_durations = data_utils.trim_rare_labels_separately(
    y_train_raw_pitches, y_train_raw_durations, 
    min_freq=MIN_FREQ, 
    unk=UNKNOWN
)

X_train, indices_cache = data_utils.prepare_input_X(
    X_train_n_raw, 
    X_train_ch_raw, 
    is_start_train, 
    row_to_song_idx_train, 
    mode=MODE,
    notes_window=NOTES_WINDOW, 
    chords_window=CHORDS_WINDOW,
    chord_size=PITCHES_PER_CHORD,
    chords_mapping=chords_mapping,
    embeddings=EMBEDDINGS,
    indices_cache=indices_cache
)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train_pitches, token_to_id_pitches = data_utils.prepare_input_y(y_train_raw_pitches, noneclass=NONECLASS)
y_train_durations, token_to_id_durations = data_utils.prepare_input_y(y_train_raw_durations, noneclass=NONECLASS)

id_to_token_pitches = {token_to_id_pitches[token] : token for token in token_to_id_pitches}
id_to_token_durations = {token_to_id_durations[token] : token for token in token_to_id_durations}

print("pitches:", y_train_pitches.shape)
print("durations:", y_train_durations.shape)

from sklearn.model_selection import train_test_split

song_idx_list = list(set(row_to_song_idx_train))
train_idx_list, validation_idx_list = train_test_split(song_idx_list, train_size=1-VALIDATION_SIZE, test_size=VALIDATION_SIZE, random_state=20202208)   

print("# TRAIN SONGS:", len(train_idx_list))
print("# VALIDATION SONGS:", len(validation_idx_list))

# _________________ NEW FEATURE = SCALING

import pandas as pd
df = pd.DataFrame(X_train.reshape(X_train.shape[0], -1))

columns_to_scale = [i for i in range(NOTES_EMB_SIZE*NOTES_WINDOW, NOTES_EMB_SIZE*NOTES_WINDOW+NOTES_WINDOW)]
off = NOTES_EMB_SIZE*NOTES_WINDOW+NOTES_WINDOW
off += CHORDS_EMB_SIZE*(CHORDS_WINDOW+1)
columns_to_scale.extend([i for i in range(off, off+CHORDS_WINDOW+1+1)])

from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
df[columns_to_scale] = mms.fit_transform(df[columns_to_scale])

X_train_reshaped = df.values.reshape(X_train.shape[0], 1, -1)

#  _________________ END OF NEW FEATURE

if not NONECLASS:
    print("NO SCALING :)")
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, -1)

Loaded TEST indices
222 1317
No. UNK chords: 0
pitches: (12070, 43)
durations: (12070, 20)
# TRAIN SONGS: 198
# VALIDATION SONGS: 22
NO SCALING :)


In [6]:
transitions = data_utils.get_bigram_transitions_chord_wise(notes_translated, chords_translated, translate_pitch_to_name=True)
with open(os.path.join(PATH, "models", "transitions", f"{ARTIST}_transitions.json"), "w") as handle:
    json.dump(transitions, handle)

In [91]:
BATCH_SIZE = 1
batch_input_shape = (BATCH_SIZE, 1, X_train.shape[1])
num_pitches = y_train_pitches.shape[-1]
num_durations = y_train_durations.shape[-1]

LSTM_SIZE = 512 if ARTIST == "*" else 128
LEARNING_RATE = 1e-5
model, opt, model_name = models.get_stateful_lstm_multi_output(batch_input_shape, num_pitches, num_durations, LSTM_SIZE, lr=LEARNING_RATE)
# model, opt, model_name = models.get_stateful_lstm_multi_output_deeper_NEW(batch_input_shape, num_pitches, num_durations, LSTM_SIZE, LSTM_SIZE, lr=LEARNING_RATE)

model_name += f"_{ARTIST}"
model_dir = os.path.join(PATH, "models", MODE, model_name)
print(model_dir)

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
predictions_dir = os.path.join(model_dir, "predictions")
if not os.path.exists(predictions_dir):
    os.makedirs(predictions_dir)


storage/models/embeddings/LSTM-512_*


In [93]:
with open(os.path.join(model_dir, "loss.json"), "r") as handle:
    loss_per_epoch = json.load(handle)
    epochs_num = len(loss_per_epoch["train"]["all"])
#     epochs_num = 50
    
try:
    model.load_weights(os.path.join(model_dir, f'weights_{epochs_num}.hdf5'))
except Exception as ex:
    print(ex)
    model.load_weights(os.path.join(model_dir, f'weights-latest.hdf5'))
epochs_num

Unable to open file (unable to open file: name = 'storage/models/embeddings/LSTM-512_*/weights_51.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


51

In [35]:
chords = chords_translated[0]

ch_idx = 0
seed_notes = []
note_t, chord_t = 0.0, 0.0
prev_note = None

TEMPERATURE = 1.0
# while len(seed_notes) < NOTES_WINDOW:
while True:
    current_chord = chords[ch_idx]
    print(current_chord)
    if not prev_note:
        tr_select = list(transitions[current_chord]["START_0.0"].items())
    else:
        tr_select = list(transitions[current_chord][prev_note].items())
        
    notes = [pair[0] for pair in tr_select]
    probs = np.array([pair[1] for pair in tr_select])
    probs /= TEMPERATURE
    probs = probs / sum(probs)
    
    next_note = np.random.choice(notes, p=probs)
    seed_notes.append(next_note)
    print(next_note)
    
    note_t += float(next_note.split('_')[-1])
    while chord_t < note_t:
        chord_t += float(chords[ch_idx].split('_')[-1])
        ch_idx += 1
        if ch_idx == len(chords):
            break
        else: 
            current_chord = chords[ch_idx]
            
    if ch_idx == len(chords):
        break
  
midi_stream = pred_utils.newsong_to_midistream(seed_notes, chords, tr_chords_dict=chords_mapping)
out_fnam = f"prob_sample_T={'None' if not TEMPERATURE else TEMPERATURE}_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}.mid"
midi_stream.write(
    'midi', 
    fp=os.path.join(predictions_dir, out_fnam)
)

A minor triad_2.0
D5_0.5
A quartal trichord_2.0
G5_0.75
A quartal trichord_2.0
G5_0.75
A quartal trichord_2.0
G5_0.75
A minor triad_2.0
G#4_0.5
A minor triad_2.0
D4_0.5
A minor triad_2.0
D5_0.5
C major triad_2.0
REST_0.25
C major triad_2.0
B6_0.5
C major triad_2.0
E5_0.75
C major triad_2.0
E6_0.5
D major triad_4.0
REST_0.25
D major triad_4.0
C4_0.75
D major triad_4.0
F5_0.5
D major triad_4.0
G6_0.5
F major triad_2.0
B-4_2.0
F major triad_2.0
REST_0.25
F major triad_2.0
A3_1.5
F major triad_2.0
G4_0.75
G major triad_2.0
C6_1.5


'storage/models/embeddings/LSTM-NEW-128-128_zun/predictions/prob_sample_T=0.2_2020-11-28_13:14:13.mid'

In [68]:
import random

# random.shuffle(validation_idx_list)
for TEMPERATURE in [None, 0.1, 0.25, 0.75, 1.0, 1.5, 2.5]:
    print(TEMPERATURE)
    songs_buffer = {}
    # for iterlist in [train_idx_list, validation_idx_list]:
    for iterlist in [validation_idx_list]:
#         random.shuffle(iterlist)
#         for i in range(len(iterlist)):
        for i in range(min(len(iterlist), 5)):
            sample_song_idx = iterlist[i]
            # ABBA: 27
            # ZUN: 86
            # all: 2508, 8285

            if sample_song_idx in train_idx_list:
                dset_type = "train"
            elif sample_song_idx in validation_idx_list:
                dset_type = "val"
            else:
                dset_type = "test"
            if TEMPERATURE:
                out_fnam = f'{sample_song_idx}_{dset_type}_t={TEMPERATURE}.json'
            else:
                out_fnam = f'{sample_song_idx}_{dset_type}_t=None.json'

            sample_chords = chords_translated[sample_song_idx]
            first, _, indices_cache = data_utils.extract_first_last_by_song_idx(sample_song_idx, row_to_song_idx_train, indices_cache=indices_cache)
            seed_n, _ = X_train_n_raw[first], X_train_ch_raw[first]
            melody = pred_utils.predict_melody(
                MODE,
                seed_n, 
                sample_chords, 
                model, 
                id_to_token_pitches, 
                id_to_token_durations, 
                embeddings=EMBEDDINGS,
                scaler=None,
                notes_window=NOTES_WINDOW, 
                chords_window=CHORDS_WINDOW, 
                chord_size=PITCHES_PER_CHORD, 
                chords_mapping=chords_mapping,
                temperature=TEMPERATURE
            )

            data = {
                "melody": melody, 
                "song": TRAIN_SONGS[sample_song_idx]
            }

            with open(os.path.join(predictions_dir, out_fnam), "w") as handle:
                json.dump(
                    data, 
                    handle
                )

            midi_stream = pred_utils.newsong_to_midistream(melody, sample_chords, tr_chords_dict=chords_mapping)
            midi_stream.write(
                'midi', 
                fp=os.path.join(predictions_dir, out_fnam.replace('.json', '.mid'))
            )

            print(TRAIN_SONGS[sample_song_idx])
#             midi_stream.measures(0, 20).plot(
#                 title=f'Sample song | Simple LSTM model trained with all artists songs\nTemperature={TEMPERATURE}'
#             )


None
datasets/pianoroll/j/jukebox-the-ghost/the-stars/chorus_nokey.mid
datasets/pianoroll/t/they-might-be-giants/weep-day/verse_nokey.mid
datasets/pianoroll/r/radiohead/pyramid-song/verse_nokey.mid
datasets/pianoroll/j/jin/yobanashi-deceive/pre-chorus_nokey.mid
datasets/pianoroll/l/lim-kim/awoo/pre-chorus_nokey.mid
0.1
datasets/pianoroll/j/jukebox-the-ghost/the-stars/chorus_nokey.mid
datasets/pianoroll/t/they-might-be-giants/weep-day/verse_nokey.mid
datasets/pianoroll/r/radiohead/pyramid-song/verse_nokey.mid
datasets/pianoroll/j/jin/yobanashi-deceive/pre-chorus_nokey.mid
datasets/pianoroll/l/lim-kim/awoo/pre-chorus_nokey.mid
0.25
datasets/pianoroll/j/jukebox-the-ghost/the-stars/chorus_nokey.mid
datasets/pianoroll/t/they-might-be-giants/weep-day/verse_nokey.mid
datasets/pianoroll/r/radiohead/pyramid-song/verse_nokey.mid
datasets/pianoroll/j/jin/yobanashi-deceive/pre-chorus_nokey.mid
datasets/pianoroll/l/lim-kim/awoo/pre-chorus_nokey.mid
0.75
datasets/pianoroll/j/jukebox-the-ghost/the-st

In [94]:
X_test_n_raw, X_test_ch_raw, y_test_raw, is_start_test, row_to_song_idx_test = data_utils.prepare_raw_X_y(
    notes_translated_test, 
    chords_translated_test,
    notes_window=NOTES_WINDOW,
    chords_window=CHORDS_WINDOW,
    use_next_chord=True,
    rep=REP
)

y_test_raw_pitches, y_test_raw_durations = data_utils._split_notes(y_test_raw)
y_test_raw_pitches, y_test_raw_durations = data_utils.trim_rare_labels_separately(
    y_test_raw_pitches, y_test_raw_durations, 
    min_freq=MIN_FREQ, 
    unk=UNKNOWN
)

indices_cache_test = {}
X_test, indices_cache_test = data_utils.prepare_input_X(
    X_test_n_raw, 
    X_test_ch_raw, 
    is_start_test, 
    row_to_song_idx_test, 
    mode=MODE,
    notes_window=NOTES_WINDOW, 
    chords_window=CHORDS_WINDOW,
    chord_size=PITCHES_PER_CHORD,
    chords_mapping=chords_mapping,
    embeddings=EMBEDDINGS,
    indices_cache=indices_cache_test
)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y_test_pitches, _ = data_utils.prepare_input_y(y_test_raw_pitches, noneclass=NONECLASS)
y_test_durations, _ = data_utils.prepare_input_y(y_test_raw_durations, noneclass=NONECLASS)

test_song_idx_list = list(set(row_to_song_idx_test))
print("# test SONGS:", len(test_song_idx_list))

No. UNK chords: 0
# test SONGS: 1307


In [95]:
print(predictions_dir)
for TEMPERATURE in [None, 0.1, 0.25, 0.75, 1.0, 1.5, 2.5, 10.0]:
    print(TEMPERATURE)
    for iterlist in [test_song_idx_list]:
    #     for i in range(len(iterlist)):
        for i in range(5):
            sample_song_idx = iterlist[i]
            sample_chords = chords_translated_test[sample_song_idx]
            first, _, indices_cache_test = data_utils.extract_first_last_by_song_idx(
                sample_song_idx, row_to_song_idx_test, indices_cache=indices_cache_test
            )
            seed_n, _ = X_test_n_raw[first], X_test_ch_raw[first]
            if not TEMPERATURE:
                print(seed_n)

            melody = pred_utils.predict_melody(
                MODE,
                seed_n, 
                sample_chords, 
                model, 
                id_to_token_pitches, 
                id_to_token_durations, 
                embeddings=EMBEDDINGS,
                scaler=None,
                notes_window=NOTES_WINDOW, 
                chords_window=CHORDS_WINDOW, 
                chord_size=PITCHES_PER_CHORD, 
                chords_mapping=chords_mapping,
                temperature=TEMPERATURE
            )
            dset_type = "test"
            
            if TEMPERATURE:
                out_fnam = f'{sample_song_idx}_{dset_type}_t={TEMPERATURE}.json'
            else:
                out_fnam = f'{sample_song_idx}_{dset_type}_t=None.json'

#             with open(os.path.join(predictions_dir, f'{sample_song_idx}_{dset_type}.json'), "w") as handle:
#                 json.dump(
#                     {"melody": melody, 
#                      "song": TEST_SONGS[sample_song_idx]
#                     }, 
#                     handle
#                 )

            midi_stream = pred_utils.newsong_to_midistream(melody, sample_chords, tr_chords_dict=chords_mapping)
            outpath = os.path.join(predictions_dir, out_fnam.replace('json', 'mid'))
            midi_stream.write(
                'midi', 
                fp=outpath
            )
            print(TEST_SONGS[sample_song_idx])


storage/models/embeddings/LSTM-512_*/predictions
None
['C4_4.0', 'REST_4.0', 'G3_1.0', 'C4_0.5', 'D4_1.0', 'E4_1.5', 'B3_2.0', 'REST_8.0']
datasets/pianoroll/a/a-great-big-world/say-something/verse_nokey.mid
['C3_4.0', 'B3_4.0', 'C3_4.0', 'G3_1.5', 'A3_1.5', 'G3_1.0', 'C3_4.0', 'G3_4.0']
datasets/pianoroll/a/a-ha/take-on-me/chorus_nokey.mid
['F3_1.5', 'F3_1.0', 'E3_0.5', 'D3_1.0', 'REST_3.5', 'C3_0.5', 'E3_0.5', 'E3_1.0']
datasets/pianoroll/a/a-ha/take-on-me/verse-and-pre-chorus_nokey.mid
['REST_1.5', 'D5_1.5', 'A4_2.0', 'D5_1.0', 'A4_3.0', 'E4_2.0', 'A4_1.0', 'F4_3.0']
datasets/pianoroll/a/a-perfect-circle/the-doomed/chorus_nokey.mid
['REST_2.0', 'G4_1.0', 'A4_0.5', 'F#4_2.5', 'REST_4.0', 'G4_0.5', 'G4_1.0', 'G4_1.5']
datasets/pianoroll/a/a-perfect-circle/the-nurse-who-loved-me/pre-chorus-and-chorus_nokey.mid
0.1
datasets/pianoroll/a/a-great-big-world/say-something/verse_nokey.mid
datasets/pianoroll/a/a-ha/take-on-me/chorus_nokey.mid
datasets/pianoroll/a/a-ha/take-on-me/verse-and-pre-

In [96]:
! tar -cf $predictions_dir/pack_test.tar $predictions_dir/*test*.mid --newer=2020-11-28